In [1]:
from transformers import AutoTokenizer
from transformers import BertTokenizer, BertForMaskedLM, BertForNextSentencePrediction,BertForQuestionAnswering
from torch.nn import functional as F
import torch
from transformers import pipeline



In [2]:
model_name = 'dccuchile/bert-base-spanish-wwm-uncased'

#Tokenizer

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Utilizar el tokenizer de forma básico (separa el texto por palabra/sub-palabra),con la función tokenize se puede tokenizar.

In [4]:
sentence = "M47 Labs es una compañía dedicada al analisis de lenguaje, esta ubicada en Barcelona,Cataluña"

In [5]:
tokenizer.tokenize(sentence)

['m',
 '##4',
 '##7',
 'lab',
 '##s',
 'es',
 'una',
 'compañía',
 'dedicada',
 'al',
 'anal',
 '##isis',
 'de',
 'lenguaje',
 ',',
 'esta',
 'ubicada',
 'en',
 'barcelona',
 ',',
 'cataluña']

Condificar el input para alimentar a un modelo

Este proceso consiste en 3 pasos primero separar el texto en palabras,sub-palabras o letras, luego se agregan los tokens especiales [cls],[sep] y por ultimo se le asigna un valor a cada token dependiendo del vocabulario

In [6]:
tokenizer.encode_plus(sentence,padding="max_length", max_length=50)

{'input_ids': [4, 1023, 1003, 998, 5004, 30958, 1028, 1091, 3663, 10639, 1074, 6634, 3838, 1009, 8023, 1019, 1149, 6616, 1035, 5779, 1019, 10373, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

inouts_ids : valor numerico del token asignado por el vocabulario.

attention_mask: vector booleano que indica si se le presta atencion a dichos tokens, en caso de tener padding asigna el valor cero al token especial [PAD]

token_type_ids: para pares de oraciones (oracion1,oracion2) indica a que oracion pertenece un token. (1,0)

Otra aplicacion es convertir los tokens_ids a palabras, este proceso se llama decod

In [7]:
decode_sentence = tokenizer.decode([4, 1023, 1003, 998, 5004, 30958, 1028, 1091, 3663, 10639, 1074, 6634, 3838, 1009, 8023, 1019, 1149, 6616, 1035, 5779, 1019, 10373, 5])

decode_sentence

'[CLS] m47 labs es una compañía dedicada al analisis de lenguaje, esta ubicada en barcelona, cataluña [SEP]'

# Aplicaciones Basicas de HF

## Mask Model: es la tarea de decodificar un token enmascarado(incognito) , rellenar los espacion en blanco.

In [8]:
tokenizer_mask = BertTokenizer.from_pretrained(model_name)

model_mask = BertForMaskedLM.from_pretrained(model_name, return_dict = True)

texto  = "La Torre Eiffel esta ubicada en " + tokenizer.mask_token + ", Francia."

# input del modelo contiene input_ids,attention_maks,token_type_ids
input = tokenizer.encode_plus(texto, return_tensors = "pt")

# ubico mi elemneto a estimar
mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)


# calculasmo los logits 
output = model_mask(**input)


logits = output.logits

#aplicamos la funcion softmax en la dimension columnas
softmax = F.softmax(logits, dim = -1)

# tomamos la matriz correspondiente a el elemento cubierto
mask_word = softmax[0, mask_index, :]

# seleccionamos los 10 token_ids con meyor ranking
top_10 = torch.topk(mask_word, 10, dim = 1)[1][0]

for token in top_10:

   word = tokenizer.decode([token])

   new_sentence = texto.replace(tokenizer.mask_token, word)

   print(new_sentence)



La Torre Eiffel esta ubicada en paris, Francia.
La Torre Eiffel esta ubicada en francia, Francia.
La Torre Eiffel esta ubicada en marsella, Francia.
La Torre Eiffel esta ubicada en cannes, Francia.
La Torre Eiffel esta ubicada en france, Francia.
La Torre Eiffel esta ubicada en ginebra, Francia.
La Torre Eiffel esta ubicada en estrasburgo, Francia.
La Torre Eiffel esta ubicada en lyon, Francia.
La Torre Eiffel esta ubicada en niza, Francia.
La Torre Eiffel esta ubicada en [UNK], Francia.


## Next Sentence Prediction: se busca si una oracion sigue a otra oracion


In [9]:
bert_tokenizer = BertTokenizer.from_pretrained(model_name)
model_nsp = BertForNextSentencePrediction.from_pretrained(model_name)
sentence_1 = "El niño vino a casa de la escuela"
sentence_2 = "Luego de la escuela,fue a su clase de natación ."
input = bert_tokenizer.encode_plus(sentence_1,sentence_2,return_tensors='pt')
outputs = model_nsp(**input)[0]
softmax = F.softmax(outputs, dim = 1)
print(softmax)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForNextSentencePrediction were not initialized from the model checkpoint at dccuc

tensor([[0.4168, 0.5832]], grad_fn=<SoftmaxBackward>)


## Zero Shot Classification: asigna una etiqueta a un texto, el input es un texto y un serie de etiquetas 

In [10]:
model_zs = "facebook/bart-large-mnli"
zs_classifier = pipeline("zero-shot-classification",
                      model =  model_zs)


sequence_to_classify = "El partido estuvo muy parajo, el primer gol fue muy bueno"
candidate_labels = ['deportes', 'politica', 'economia']
zs_classifier(sequence_to_classify, candidate_labels)




{'sequence': 'El partido estuvo muy parajo, el primer gol fue muy bueno',
 'labels': ['politica', 'economia', 'deportes'],
 'scores': [0.45973241329193115, 0.2714166045188904, 0.2688509225845337]}

## Question Answering: el modelo reponde una pregunta basandose en un contexto

In [11]:
from transformers import BertTokenizer, BertForQuestionAnswering

model_name_qs = 'mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es'
model_qa = BertForQuestionAnswering.from_pretrained(model_name_qs)
tokenizer = BertTokenizer.from_pretrained(model_name_qs)

pregunta = 'Donde trabaj enrique?'

texto = "Enrique es ingeniero mecanico y trabaja en Mclaren"

# se debe alimentar el  tokenizer con la pregunta  y el texto
encoding = tokenizer.encode_plus(pregunta, texto,return_tensors='pt')
tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])

# se alimenta el modelo con el texto codificado
output = model_qa(**encoding)



answer_start = torch.argmax(output.start_logits)

answer_end = torch.argmax(output.end_logits)

if answer_end >= answer_start:
     answer = tokens[answer_start]
     for i in range(answer_start+1, answer_end+1):
         if tokens[i][0:2] == "##":
             answer += tokens[i][2:]
         else:
             answer += " " + tokens[i]
            
if answer.startswith("[CLS]"):
     answer = "Unable to find the answer to your question."

print("\nPredicted answer:\n{}".format(answer.capitalize()))


Predicted answer:
Mclaren
